In [1]:
import sys
import argparse
import cv2
import shutil
import pickle
import json
import glob
from scipy.io import loadmat
import numpy as np
import os
import tqdm


import torch, torchvision
print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose
print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())


from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results,inference_top_down_pose_model_modified
                        ,vis_pose_result_modified)
from mmdet.apis import inference_detector, init_detector
local_runtime = False


from torchvision.transforms import transforms
import time

import numpy as np
from IPython.display import Image, display
import tempfile
import os.path as osp


categories = [{'supercategory': 'animal',
  'id': 1,
  'name': 'animal',
  'keypoints': ['bottom_trunk',
   'mid_trunk',
   'top_trunk',
   'bottom_right_tusk',
   'bottom_left_tusk',
   'top_right_tusk',
   'top_left_tusk',
   'right_eye',
   'left_eye',
   'right_bottom_ear',
   'left_bottom_ear',
   'right_bottom_tip_ear',
   'left_bottom_tip_ear',
   'right_side_tip_ear',
   'left_side_tip_ear',
   'top_right_ear',
   'top_left_ear',
   'top_right_tip_ear',
   'top_left_tip_ear',
   'hoof',
   'tail',
   'right_front_elbow',
   'left_front_elbow',
   'right_back_elbow',
   'left_back_elbow',
   'right_front_knee',
   'left_front_knee',
   'right_back_knee',
   'left_back_knee',
   'right_front_foot',
   'left_front_foot',
   'right_back_foot',
   'left_back_foot'],
  'skeleton': [[1, 2],
   [2, 3],
   [3, 8],
   [3, 9],
   [8, 9],
   [8, 16],
   [9, 17],
   [3, 10],
   [3, 11],
   [10, 20],
   [11, 20],
   [10, 22],
   [11, 22],
   [10, 23],
   [11, 23],
   [22, 26],
   [23, 27],
   [26, 30],
   [27, 31],
   [20, 21],
   [21, 24],
   [21, 25],
   [24, 28],
   [25, 29],
   [28, 32],
   [29, 33]]}]
   
   
   
   
# initialize pose model

#pose_config = '/media/obafemi/New Volume/ViTPose/vitpose_b_elephant.py'
#pose_config = '/media/obafemi/New Volume/ViTPose/vitpose_b_femi_animalpose.py'
#pose_checkpoint = '/media/obafemi/New Volume/ViTPose/work_dirs/vitpose_b_femi_animalpose/best_AP_epoch_200.pth'


pose_config = '/media/obafemi/New Volume/ViTPose/vitpose_b_elephant_teacher.py'
pose_checkpoint = '/media/obafemi/New Volume/ViTPose/work_dirs/vitpose_b_elephant_teacher_lr_reduced earlier/best_AP_epoch_330.pth'
det_config = 'mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'




pose_model = init_pose_model(pose_config, pose_checkpoint)
# initialize detector

det_model = init_detector(det_config, det_checkpoint)
pseudo = '/media/obafemi/New Volume/ViTPose/data/elephant_pseudo/data/train2017'
#pseudo = '/media/obafemi/New Volume/ViTPose/demo/rhino'

#classes = os.listdir('/media/obafemi/New Volume/ViTPose/unlabeled_dataset')
#classes= ['elephants']

category_id = 0
is_crowd = 0


pseudo_image_list = []
pseudo_annotations = []

M = 0

imgs_in = os.listdir(pseudo)

for i, im_name in enumerate(imgs_in):
   
    image_name_ = imgs_in[i]
    image_name = os.path.join(pseudo, image_name_)

    img = cv2.imread(image_name)
    if img is None:
        print(image_name)
    h, w, _ = img.shape
    #if h * w < 256 * 256:
     #   continue
    mmdet_results = inference_detector(det_model, img)
    
    final_det = []
   
    person_results = process_mmdet_results(mmdet_results, cat_id=21)

    pose_results, returned_outputs = inference_top_down_pose_model_modified(pose_model,
                                                                   img,
                                                                   person_results,
                                                                   bbox_thr=0.3,
                                                                   format='xyxy',
                                                                    cat_id=0
                                                                           )#dataset=pose_model.cfg.data.test.type)


/home/obafemi/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


torch version: 1.7.1+cu110 True
torchvision version: 0.8.2+cu110
mmpose version: 0.24.0
cuda version: 11.7
compiler information: GCC 9.4
apex is not installed
apex is not installed
apex is not installed
Use load_from_local loader


/home/obafemi/anaconda3/lib/python3.9/site-packages/mmdet/core/anchor/builder.py:15: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  warnings.warn(


Use load_from_http loader


/home/obafemi/anaconda3/lib/python3.9/site-packages/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/home/obafemi/anaconda3/lib/python3.9/site-packages/mmdet/core/anchor/anchor_generator.py:359: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  warnings.warn(


KeyboardInterrupt: 

In [12]:
import sys
sys.path.insert(0, '/media/obafemi/New Volume/Evaluate_animal_pose_with_different_models')
import os
import argparse
import cv2
import shutil
import pickle
import json
import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt
from helper_functions import *
from generateReal import *
from generateSynthetic import *
from generatePseudos import *

# Check Pytorch installation
import torch, torchvision
print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose
print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

import cv2
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results,inference_top_down_pose_model_modified
                        ,vis_pose_result_modified)
from mmdet.apis import inference_detector, init_detector
local_runtime = False


import os
import cv2
import numpy as np
from numpy import random
import torch
from torchvision.transforms import transforms
import time

from models.hrnet import HRNet
from models.poseresnet import PoseResNet

from models.experimental import attempt_load
from utils.datasets import letterbox
from utils.general import check_img_size, non_max_suppression, scale_coords, xyxy2xywh
from utils.torch_utils import select_device, time_synchronized
from utils.inference import *
from config import cfg
from config import update_config

import numpy as np
from IPython.display import Image, display
import tempfile
import os.path as osp

torch version: 1.7.1+cu110 True
torchvision version: 0.8.2+cu110
mmpose version: 0.24.0
cuda version: 11.7
compiler information: GCC 9.4


In [13]:
 def get_bboxes(img0, weights='/media/obafemi/New Volume/Evaluate_animal_pose_with_different_models/yolo_weights/yolov5x.pt', classes=list(range(0, 80))):

        imgsz = 640
        device = '0'

        # Initialize
        device = select_device(device)  # make new output folder

        # Load model
        model = attempt_load(weights, map_location=device)  # load FP32 model
        imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size

        half = False
        # Get names and colors
        names = model.module.names if hasattr(model, 'module') else model.names
        #class_map = names.index(classes)
        a = dict(zip(names, np.arange(80)))
        class_map = list(a.keys())
        #print(class_map)
        #class_map = classes
        # Run inference
        t0 = time.time()
        img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
        _ = model(img.half() if half else img) if device.type != 'cpu' else None
        bboxes = []  # run once

        img = letterbox(img0, new_shape=imgsz)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img = img.float()
        img /= 255.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

            # Inference
        t1 = time_synchronized()
        pred = model(img, augment=True)[0]

        # Apply NMS
        pred = non_max_suppression(pred, 0.6, 0.5, classes=classes, agnostic=True)  # opt.classes
        t2 = time_synchronized()

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            s, im0 = '', img0
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
                #print(det[:,-1].item()) #############################
                # Print results
                #print(det)
                for c in det[:, -1]:
                    n = (det[:, -1] == c).sum()  # detections per class
                    #print(n)
                    s += '%g %ss, ' % (n, names[int(c)])  # add to string

                # Append results
                for *xyxy, conf, cls in det:
                    #print(,conf,cls)
                    #f cls == class_map:
                     #   print('here')
                    dic = {'bbox':np.array([xyxy[0].item(), xyxy[1].item(), xyxy[2].item(),xyxy[3].item(),conf.item()],dtype = 'float32'),'cls':np.array(cls.item())}
                    #bboxes.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]),conf.item()])
                    bboxes.append(dic)
                    #bboxes.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]),conf.item()])

        return bboxes
    
def get_catID(cls):
    cat_id = 0 if cls == 20 else 1
    
    return cat_id

In [14]:
import sys
import argparse
import cv2
import shutil
import pickle
import json
import glob
from scipy.io import loadmat
import numpy as np
import os
import tqdm


import torch, torchvision
print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose
print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())


from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results,inference_top_down_pose_model_modified
                        ,vis_pose_result_modified)
from mmdet.apis import inference_detector, init_detector
local_runtime = False


from torchvision.transforms import transforms
import time

import numpy as np
from IPython.display import Image, display
import tempfile
import os.path as osp

# initialize pose model
import numpy as np
import os
import tqdm
import json
import shutil


def process_mmdet_results_modified(mmdet_results):
    """Process mmdet results, and return a list of bboxes.

    Args:
        mmdet_results (list|tuple): mmdet results.
        cat_id (int): category id (default: 1 for human)

    Returns:
        person_results (list): a list of detected bounding boxes
    """
    if isinstance(mmdet_results, tuple):
        det_results = mmdet_results[0]
    else:
        det_results = mmdet_results
    
    det_results.pop(20)
    det_results.pop(0)
    
    bboxes = np.zeros((1,5))
    
    for i in range(len(det_results)):
        bboxes = np.vstack((bboxes,det_results[i]))
        
    #bboxes = det_results[0]
    bboxes = bboxes[1:,:]
    person_results = []
    for bbox in bboxes:
        person = {}
        person['bbox'] = bbox
        person_results.append(person)

    return person_results
    
    
 

pose_config = '/media/obafemi/New Volume/ViTPose/ViTPose_large_ap10k_256x192.py'
pose_checkpoint = '/media/obafemi/New Volume/ViTPose/work_dirs/epoch_500.pth'
det_config = 'mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'




pose_model = init_pose_model(pose_config, pose_checkpoint)
# initialize detector

det_model = init_detector(det_config, det_checkpoint)
pseudo = '/media/obafemi/New Volume/ViTPose/Deer Dataset Sample/new data/image'



imgs_in = os.listdir(pseudo)
#print(imgs_in)
for i, im_name in enumerate(imgs_in):
    #print(i, im_name)

    image_name_ = imgs_in[i]
    image_name = os.path.join(pseudo, image_name_)
 
    img = cv2.imread(image_name)

    h, w, _ = img.shape
    #if h * w < 256 * 256:
    # continue
    mmdet_results = inference_detector(det_model, img)
    person_results = process_mmdet_results_modified(mmdet_results)
    
    
    #person_results = process_mmdet_results(mmdet_results, cat_id=0)
    final_det = get_bboxes(img)
    person_results = final_det

    if len(person_results)==0:
        continue
    pose_results, returned_outputs = inference_top_down_pose_model_modified(pose_model,
                                                                img,
                                                                person_results,
                                                                bbox_thr=0.3,
                                                                format='xyxy',
                                                                    cat_id=1
                                                                        )

     # show pose estimation results
    vis_result = vis_pose_result_modified(pose_model,
                                 img,
                                pose_results,
                                          cat_id=1,
                                          #cat_id=1,
                                          kpt_score_thr=0.42,
                                #dataset=pose_model.cfg.data.test.type,
                                 show=False)
    # reduce image size
     # reduce image size
    vis_result = cv2.resize(vis_result, dsize=None, fx=0.5, fy=0.5)



    #with tempfile.TemporaryDirectory() as tmpdir:
    file_name = osp.join('/media/obafemi/New Volume/ViTPose/demo/deer_poses', 'pose_results'+str(i)+'.png')
    #print(file_name)
    cv2.imwrite(file_name, vis_result)
    #display(Image(file_name))




torch version: 1.7.1+cu110 True
torchvision version: 0.8.2+cu110
mmpose version: 0.24.0
cuda version: 11.7
compiler information: GCC 9.4
Use load_from_local loader
Use load_from_http loader


In [15]:
inference_top_down_pose_model_modified?

In [16]:
vis_pose_result_modified?